## API Setup

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
from langchain_groq import ChatGroq
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import bs4
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage,HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Embedding Model

In [3]:
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

## LLM model

In [ ]:

llm=ChatGroq(model_name="llama3-8b-8192")

## Data Loding

In [5]:

loader=WebBaseLoader(
    web_path=("https://www.amplework.com/",),)

In [6]:
data=loader.load()

In [44]:
print(data)

[Document(metadata={'source': 'https://www.amplework.com/', 'title': 'Top AI Development Agency in USA | Expert AI Developers', 'description': 'Amplework Software, A top AI development agency, offers custom AI solutions & AI agent development by full-stack developers. Schedule a consultation today!', 'language': 'en-US'}, page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTop AI Development Agency in USA | Expert AI Developers\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWho we are\n\n\n\n\n\nAmplework Software: Transforming Success with Ai\nWe Specialize in delivering Ai-driven solutions that transform businesses, enhance processes, and drive innovation. Discover how our expertise can elevate your business to new heights.\n\nAbout Us\nCareer\nBlog\nContact\n \n\n\nOur Achievements :  Delivering Success for Clients\n\n\n30%\nAverage Increase in

Dividing Chunks

In [7]:
text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text_split.split_documents(data)

## Store vectore

In [8]:
db=Chroma.from_documents(documents=docs,embedding=embedding)


In [9]:
retriver=db.as_retriever()

## Prompt

### Adding chat History

In [45]:
store={}
contextualize_q_system_prompt=(
    "You are an AI assistant for AmpleWork soft. pvt. ltd.. Your job is to reframe user queries that may reference previous context in the conversation. Given a chat history and the latest user question, rewrite the question in a way that makes it self-contained and understandable without needing the chat history."

"Do NOT answer the question, only rephrase it. If the question is already clear on its own, return it as is."

            "Given a chat history and the latest user question"
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )
contextualize_q_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", contextualize_q_system_prompt),
                    MessagesPlaceholder("chat_history"),
                    ("human", "{input}"),
                ]
            )

history_aware_retriever=create_history_aware_retriever(llm,retriver,contextualize_q_prompt)

        ## Answer question

        # Answer question
system_prompt = (
   """You are an AI assistant for AmpleWork soft. pvt. ltd., dedicated to helping customers with their queries. Use the provided context from the company knowledge base to answer the question accurately and concisely. 

- If you don't know the answer, say 'I don’t know' instead of making something up.
- Keep responses polite, professional, and under three sentences.
- If necessary, guide the customer on the next steps they should take.
- Use friendly and clear language to enhance customer satisfaction.

Relevant company context:
{context}"""

                "You are an Amplework soft. pvt.ltd. assistant for Help to customer resolving query. "
                "Use the following pieces of retrieved context to answer "
                "the question. If you don't know the answer, say that you "
                "don't know. Use three sentences maximum and keep the "
                "answer concise."
                "\n\n"
                "{context}"
            )
qa_prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", system_prompt),
                    MessagesPlaceholder("chat_history"),
                    ("human", "{input}"),
                ]
            )
        
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        
    return store[session_id]    


conversational_rag_chain=RunnableWithMessageHistory(
            rag_chain,get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer"
        )

In [46]:
response = conversational_rag_chain.invoke(
    {"input": "who are you"},
    config={"configurable": {"session_id": "chat1"}}
)

In [47]:
response['answer']

"Hello! I am an AI assistant for AmpleWork soft. pvt.ltd., here to help you with any queries or concerns you may have. I'm dedicated to providing accurate and helpful information to ensure a smooth and satisfying experience for our customers."

In [48]:
response = conversational_rag_chain.invoke(
    {"input": "where compenay location"},
    config={"configurable": {"session_id": "chat1"}}
)
response['answer']

"According to the company's website, AmpleWork soft. pvt.ltd. has multiple location contact numbers:\n\n* (+1) 334-230-5010 (for North America)\n* (+44) 208-638-7939 (for the UK)\n* (+91) 963-696-2228 (for India)\n\nPlease feel free to reach out to any of these numbers for further assistance or inquiries."

In [49]:
response = conversational_rag_chain.invoke(
    {"input": "my name is aditya"},
    config={"configurable": {"session_id": "chat1"}}
)
response['answer']

"Nice to meet you, Aditya! I'm here to help you with any questions or concerns you may have about AmpleWork soft. pvt.ltd. or our services. How can I assist you today?"

In [50]:
response = conversational_rag_chain.invoke(
    {"input": "how i contant it"},
    config={"configurable": {"session_id": "chat1"}}
)
print(response['answer'])

Aditya, you can contact AmpleWork soft. pvt.ltd. through the following numbers or email:

* Phone numbers: (+1) 334-230-5010 (for North America), (+44) 208-638-7939 (for the UK), or (+91) 963-696-2228 (for India)
* Email: hr@amplework.com (for Human Resources inquiries) or the general email address available on our website.

Please feel free to reach out to us through whichever method is most convenient for you.


In [51]:
response = conversational_rag_chain.invoke(
    {"input": "what is java"},
    config={"configurable": {"session_id": "chat1"}}
)
print(response['answer'])

Aditya! Java is a high-level, object-oriented programming language developed by Sun Microsystems (now owned by Oracle Corporation). It's widely used for developing cross-platform applications, including Android apps, web applications, and enterprise software.

Java is known for its platform independence, meaning that programs written in Java can run on any device that has a Java Virtual Machine (JVM) installed, without modification. This makes Java a popular choice for developers who need to create applications that can run on multiple platforms.

At AmpleWork soft. pvt.ltd., we have expertise in Java development services, including Java consulting, Java development, and Java testing. If you have any specific questions or requirements related to Java, feel free to ask!


In [52]:
response = conversational_rag_chain.invoke(
    {"input": "who is ceo?"},
    config={"configurable": {"session_id": "chat1"}}
)
print(response['answer'])

Aditya, I'm not aware of the CEO's information as it's not publicly available on our website. However, I can suggest checking our About Us page or contacting our HR department (hr@amplework.com) for more information on our leadership team.
